In [8]:
import json
import nltk
import numpy as np
import pickle
import random
import tensorflow as tf
import tflearn

# Load intents data
def load_intents_data(filename):
    with open(filename) as file:
        data = json.load(file)
    return data

# Preprocess data
def preprocess_data(data):
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            # Tokenize and stem each word
            words.extend(nltk.word_tokenize(pattern))
            docs_x.append(nltk.word_tokenize(pattern))
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training, output = [], []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = [1 if stemmer.stem(w) in doc else 0 for w in words]
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    print("Preprocessed words:", words)
    print("Labels:", labels)
    print("Training data:", training)
    print("Output data:", output)

    return words, labels, training, output

# Save preprocessed data
def save_preprocessed_data(data, filename):
    with open(filename, "wb") as file:
        pickle.dump(data, file)

# Load preprocessed data
def load_preprocessed_data(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

# Define the neural network model
def create_model(training, output):
    tf.compat.v1.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)

    model = tflearn.DNN(net)
    return model

# Train the model
def train_model(model, training, output, model_filename):
    try:
        model.load(model_filename)
        print("Trained model loaded from", model_filename)
    except:
        model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
        model.save(model_filename)
        print("Model trained and saved to", model_filename)

# Convert input sentence into a bag of words representation
def convert_to_bag_of_words(sentence, words):
    bag = [0 for _ in range(len(words))]
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]

    for i, w in enumerate(words):
        if w in sentence_words:
            bag[i] = 1

    return np.array(bag)

# Chat function for user interaction
def chat_with_bot(data, model, words, labels):
    print("Start chatting with the bot... (type quit to stop)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break

        results = model.predict([convert_to_bag_of_words(user_input, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for intent in data["intents"]:
            if intent["tag"] == tag:
                responses = intent["responses"]

        print(random.choice(responses))
    
# Main function
def main():
    intents_data = load_intents_data("intents.json")
    print("Loaded intents data from intents.json")

    try:
        words, labels, training, output = load_preprocessed_data("data.pickle")
        print("Preprocessed data loaded from data.pickle")
    except:
        words, labels, training, output = preprocess_data(intents_data)
        save_preprocessed_data((words, labels, training, output), "data.pickle")
        print("Preprocessed data saved to data.pickle")

    model = create_model(training, output)
    train_model(model, training, output, "model.tflearn")

    chat_with_bot(intents_data, model, words, labels)

if __name__ == "__main__":
    main()


Loaded intents data from intents.json
Preprocessed data loaded from data.pickle
INFO:tensorflow:Restoring parameters from c:\Users\Hp\Documents\AI\Chatbot\model.tflearn
Trained model loaded from model.tflearn
Start chatting with the bot... (type quit to stop)
Good bye
Have a nice time
Good to see you again!
Our company is situated in 'street name', Addis Ababa
